In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns


# Make numpy printouts easier to read.
np.set_printoptions(precision=3, suppress=True)
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing
zzzz
print(tf.__version__)
prime1=[]
# import pandas as pd 
def SieveOfEratosthenes(n): 
       
    # Create a boolean array "prime[0..n]" and  
    # initialize all entries it as true. A value  
    # in prime[i] will finally be false if i is 
    # Not a prime, else true. 
    prime = [True for i in range(n+1)] 
      
    p = 2
    while(p * p <= n): 
           
        # If prime[p] is not changed, then it is  
       # a prime 
        if (prime[p] == True): 
               
            # Update all multiples of p 
            for i in range(p * p, n + 1, p): 
                prime[i] = False
        p += 1
    c = 0
  
    # Print all prime numbers 
    for p in range(2, n): 
        if prime[p]: 
            prime1.append(p)
    return prime1
  
prime=SieveOfEratosthenes(4000000)

num=[]
newlist=[]
for i in range(len(prime)):
    s=prime[:i]
    newlist.append(sum(s))
    num.append(i)
 
 
  
# Calling DataFrame constructor after zipping 
# both lists, with columns specified 
dataset = pd.DataFrame(list(zip(num, prime1)), 
               columns =['num', 'sop']) 

train_dataset = dataset.sample(frac=0.8, random_state=0)
test_dataset = dataset.drop(train_dataset.index)
train_dataset.describe().transpose()


2.3.0


,count,mean,std,min,25%,50%,75%,max
num,226517.0,1.414902e+05,8.175235e+04,0.0,70722.0,141384.0,212393.0,283145.0
sop,226517.0,1.921880e+06,1.169987e+06,2.0,892261.0,1890593.0,2934143.0,3999971.0


In [ ]:
train_features = train_dataset.copy()
test_features = test_dataset.copy()

train_labels = train_features.pop('sop')
test_labels = test_features.pop('sop')

In [ ]:
normalizer = preprocessing.Normalization()
normalizer.adapt(np.array(train_features))
print(normalizer.mean.numpy())


[141490.2]


In [ ]:
horsepower = np.array(train_features['num'])

horsepower_normalizer = preprocessing.Normalization(input_shape=[1,])
horsepower_normalizer.adapt(horsepower)

In [ ]:
horsepower_model = tf.keras.Sequential([
    horsepower_normalizer,
    layers.Dense(units=1)
])

horsepower_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
normalization_1 (Normalizati (None, 1)                 3         
_________________________________________________________________
dense (Dense)                (None, 1)                 2         
Total params: 5
Trainable params: 2
Non-trainable params: 3
_________________________________________________________________


In [ ]:
horsepower_model.compile(
    optimizer=tf.optimizers.Adam(learning_rate=0.1),
    loss='mean_absolute_error')

In [ ]:
%%time
history = horsepower_model.fit(
    train_features['num'], train_labels,
    epochs=100,
    # suppress logging
    verbose=0,
    # Calculate validation results on 20% of the training data
    validation_split = 0.2)

CPU times: user 7min 23s, sys: 49.1 s, total: 8min 12s
Wall time: 6min 8s


In [ ]:
def build_and_compile_model(norm):
  model = keras.Sequential([
      norm,
      layers.Dense(100, activation='relu'),
      layers.Dense(100, activation='relu'),
      layers.Dense(1)
  ])

  model.compile(loss='mean_absolute_error',
                optimizer=tf.keras.optimizers.Adam(0.0001))
  return model
dnn_horsepower_model = build_and_compile_model(horsepower_normalizer)
dnn_horsepower_model.summary()

dnn_horsepower_model.fit(
    train_features['num'], train_labels,
    validation_split=0.2,
    verbose=1, epochs=250)


In [ ]:
print(len(prime))

283146


In [ ]:
a=[1,]
dnn_horsepower_model.predict(a)

array([[805.691]], dtype=float32)